In [3]:
import pandas as pd
import numpy as np
from unidecode import unidecode

import nltk
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import OneClassSVM

sns.set()
sns.set_style('darkgrid')
sns.set_palette('deep')


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmarques\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jmarques\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
ncm_code = '01'
data = pd.read_csv('nfedataNCM.csv', low_memory=False)
mask = data.loc[:, 'produto_ncm_capitulo'] == str(ncm_code)
data = data.loc[mask, :]
data['produto_ncm_valido'] = np.zeros((data.shape[0],))

In [7]:
desc = data.loc[:, 'produto_descricao']
desc = desc.apply(lambda x: str(x).lower())
desc = desc.apply(unidecode)
desc = desc.apply(lambda x: ' '.join(
    [
        w for w in word_tokenize(x) 
        if w not in stopwords.words('portuguese')
        ]
    )
)
desc = desc.apply(lambda x: re.sub(r'[0-9]{3,}', '', x))

In [ ]:
cv = CountVectorizer(ngram_range=(1, 2))
desc_tf = cv.fit_transform(desc)

svm = OneClassSVM()
desc_val = svm.fit_predict(desc_tf)
data.loc[:, 'produto_ncm_valido'] = desc_val

In [ ]:
data.to_csv(f'dataNCM{ncm_code}.csv', index=False)